In [1]:
#import spark
from pyspark.sql import SparkSession
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 ass2 BNPL group 28")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/13 13:18:12 WARN Utils: Your hostname, DESKTOP-ISE3FUM resolves to a loopback address: 127.0.1.1; using 172.30.155.74 instead (on interface eth0)
22/09/13 13:18:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/13 13:18:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
curated_csv = spark.read.options(header = True).csv('../data/curated/full_data.csv')

In [3]:
curated_csv.limit(5)
print(curated_csv.count(),len(curated_csv.columns))

14195505 15


In [4]:
curated_csv1 = curated_csv.drop(curated_csv.order_id)
curated_csv2 = curated_csv1.drop(curated_csv.address)
curated_csv3 = curated_csv2.drop(curated_csv.gender)
#curated_csv4 = curated_csv3.drop(curated_csv.user_name)
curated_csv3.limit(5)
print(curated_csv3.count(),len(curated_csv3.columns))

14195505 12


In [5]:
# check null value
curated_csv3 = curated_csv3.dropna()
print(curated_csv3.count(),len(curated_csv3.columns))

13614675 12


In [6]:
curated_csv3.limit(5)

user_id,merchant_abn,dollar_value,order_datetime,consumer_id,merchant_name,user_name,state,postcode,field,revenue_level,take_rate
2979,87639550697,76.55,2021-08-20,257,Mauris Consulting,Jason Newton,VIC,3385,digital goods: bo...,b,3.27
2979,80191930269,412.46,2021-08-20,257,Dolor Quisque Tin...,Jason Newton,VIC,3385,computer programm...,b,3.68
2979,69703285964,12.15,2022-05-06,257,Suspendisse Incor...,Jason Newton,VIC,3385,"cable, satellite,...",a,5.77
2979,92065881715,681.52,2021-08-20,257,Quam Elementum Corp.,Jason Newton,VIC,3385,"opticians, optica...",b,4.2
2979,63290521567,13.87,2022-05-06,257,Vehicula Pellente...,Jason Newton,VIC,3385,artist supply and...,a,6.48


In [7]:
# check wrong data
from os import curdir
from pyspark.sql import functions as F
new_data = curated_csv3.filter(
    (F.col('user_id') > 0 )
    & (F.col('dollar_value') >= 0)
    & (F.col('consumer_id') > 0)
    & (F.col('postcode') > 0)
)
print(new_data.count(),len(new_data.columns))

13614675 12


In [8]:
new_data.limit(5)

user_id,merchant_abn,dollar_value,order_datetime,consumer_id,merchant_name,user_name,state,postcode,field,revenue_level,take_rate
2979,87639550697,76.55,2021-08-20,257,Mauris Consulting,Jason Newton,VIC,3385,digital goods: bo...,b,3.27
2979,80191930269,412.46,2021-08-20,257,Dolor Quisque Tin...,Jason Newton,VIC,3385,computer programm...,b,3.68
2979,69703285964,12.15,2022-05-06,257,Suspendisse Incor...,Jason Newton,VIC,3385,"cable, satellite,...",a,5.77
2979,92065881715,681.52,2021-08-20,257,Quam Elementum Corp.,Jason Newton,VIC,3385,"opticians, optica...",b,4.2
2979,63290521567,13.87,2022-05-06,257,Vehicula Pellente...,Jason Newton,VIC,3385,artist supply and...,a,6.48


In [9]:
# check repetation 
new_data1 = new_data.distinct()
print(new_data1.count(),len(new_data1.columns))

13614664 12


In [10]:
# check the order datetime
from pyspark.sql import functions as F
new_df = new_data1.filter(
    (F.col('order_datetime') >= '2021-02-28') 
    & (F.col('order_datetime') <= '2022-08-28')
)

print(new_df.count(),len(new_df.columns))

12047306 12


In [11]:
new_df.select('postcode').describe()
#new_df.limit(5)

summary,postcode
count,12047306
mean,4043.344197117596
stddev,1780.6994322233006
min,1001
max,9999


In [12]:
# check postcode
new_df = new_df.withColumn("length_of_str",F.length('postcode'))
new_df.select('length_of_str').describe()
new_df = new_df.filter(
    (F.col('length_of_str') <= 4)
    & (F.col('length_of_str') >=3)
)
new_df = new_df.drop('length_of_str')
print(new_df.count(),len(new_df.columns))

12047306 12


In [13]:
new_df.select('postcode').describe()

summary,postcode
count,12047306
mean,4043.344197117596
stddev,1780.6994322233006
min,1001
max,9999


In [14]:
# check postcode
new_df = new_df.withColumn("length_of_str",F.length('postcode'))
new_df.select('length_of_str').describe()
new_df = new_df.filter(
    (F.col('length_of_str') == 4)
    #&(F.col('length_of_str') >=3 )
)
new_df = new_df.drop('length_of_str')
print(new_df.count(),len(new_df.columns))

11873770 12


In [15]:
new_df.select('postcode').describe()

summary,postcode
count,11873770
mean,4090.159621164971
stddev,1750.7134067860632
min,1001
max,9999


In [16]:
# check state
cur_df = new_df.filter(
    (F.col('state') == 'NT')
    | (F.col('state') == 'NSW')
    | (F.col('state') == 'QLD')
    | (F.col('state') == 'SA')
    | (F.col('state') == 'TAS')
    | (F.col('state') == 'VIC')
    | (F.col('state') == 'WA')
    | (F.col('state') == 'ACT')
)
print(cur_df.count(),len(cur_df.columns))


11873770 12
